<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین چهارم (دسته‌بندی اخبار)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        دسترسی به داده‌ها و مدل‌های ذخیره شده
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        به دلیل حجم بالای فایل‌های ذخیره شده، از قرار دادن آن‌ها در پوشه نوتبوک صرف نظر کرده و با اجرای قطعه کد زیر، از طریق گوگل درایو به آن‌ها دسترسی پیدا می‌کنیم.
    </p>
</div>

In [2]:
import sys
from google.colab import drive
drive.mount('/content/drive')
address = "University/Term 6/MIR/Homeworks/HW3"
sys.path.append(f"/content/drive/My Drive/{address}")
%cd /content/drive/My\ Drive/$address

Mounted at /content/drive
/content/drive/My Drive/University/Term 6/MIR/Homeworks/HW3


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [1]:
! pip install bs4
! pip install tqdm
! pip install pandas
! pip install requests
! pip install hazm
! pip install unidecode
! pip install datasets
! pip install scikit-learn
! pip install nltk
! pip install networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 5.3 MB/s 
     |████████████████████████████████| 1.4 MB 59.9 MB/s 
     |████████████████████████████████| 233 kB 53.0 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=c1a279c3f8642971edbbba79b58b74740ae6782794a80a78bee698de60b46e98
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64

In [6]:
import os
import re
import csv
import hazm
import nltk
import json
import torch
import pickle
import zipfile
import requests
import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm
from pathlib import Path
from datasets import Dataset
from bs4 import BeautifulSoup
from string import punctuation
from IPython.display import display
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٦٨ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [7]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

CATEGORIES_CLASSES = {
    'Politics': 0,
    'World': 1,
    'Economy': 2,
    'Society': 3,
    'City': 4,
    'Sport': 5,
    'Science': 6,
    'Culture': 7,
    'IT': 8,
    'LifeSkills': 9,
}

CLASSES_CATEGORIES = {v: k for k, v in CATEGORIES_CLASSES.items()}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [8]:
class Scraper:

    def __init__(self, current_year, current_month):
        self.current_year = current_year
        self.current_month = current_month

    def get_URL_content(self, URL):
        while True:
            try:
                return requests.get(URL, timeout=5).content
            except:
                pass

    def generate_page_URL(self, page_index, category, year, month):
        tp = {'Politics': 6, 'World': 11, 'Economy': 10, 'Society': 5, 'City': 7,
              'Sport': 9, 'Science': 20, 'Culture': 26, 'IT': 718, 'LifeSkills': 21}[category]
        return f'https://www.hamshahrionline.ir/archive?pi={page_index}&tp={tp}&ty=1&ms=0&mn={month}&yr={year}'

    def get_page_URLs_by_time(self, category, year, month):
        URLs = []
        page_index = 1
        while True:
            URL = self.generate_page_URL(page_index, category, year, month)
            content = self.get_URL_content(URL)
            if re.findall('pagination', str(content)):
                URLs.append(URL)
                page_index += 1
            else:
                break
        return URLs

    def get_page_URLs_since(self, category, year, month):
        URLs = []
        with tqdm() as pbar:
            while True:
                if month > 12:
                    month = 1
                    year += 1
                pbar.set_description(f'[{category}] [Extracting page URLs] [Date: {year}/{month}]')
                URLs_by_time = self.get_page_URLs_by_time(category, year, month)
                if URLs_by_time:
                    for URL in URLs_by_time:
                        URLs.append(URL)
                    month += 1
                elif self.current_year > year or (self.current_year == year and self.current_month > month):
                    month += 1
                else:
                    break
        return URLs

    def get_news_URLs_since(self, category, year, month):
        news_URLs = []
        page_URLs = self.get_page_URLs_since(category, year, month)
        with tqdm(page_URLs) as pbar:
            for page_URL in pbar:
                content = self.get_URL_content(page_URL)
                soup = BeautifulSoup(content, 'html5lib')
                for item in soup.findAll('li', attrs={'class': 'news'}):
                    URL = item.find('div', attrs={'class': 'desc'}).find('h3').find('a')['href']
                    URL = 'https://www.hamshahrionline.ir' + URL
                    news_URLs.append(URL)
                pbar.set_description(f'[{category}] [Extracting news URLs] [{len(news_URLs)} news until now]')
        return news_URLs

    def parse_news(self, URL, category):
        try:
            content = self.get_URL_content(URL)
            soup = BeautifulSoup(content, 'html.parser')
            date = soup.find('div', {'class': 'col-6 col-sm-4 col-xl-4 item-date'}).span.text.strip()
            title = soup.find('div', {'class': 'item-title'}).h1.text.strip()
            intro = soup.find('p', {'class': 'introtext', 'itemprop': 'description'}).text.strip()
            body = soup.find('div', {'class': 'item-text', 'itemprop': 'articleBody'}).text.strip()
            return {
                'date': date,
                'title': title,
                'intro': intro,
                'body': body,
                'category': category,
            }
        except:
            return None

    def scrape(self, from_year, from_month):
        categories = ['Politics', 'World', 'Economy', 'Society', 'City',
                      'Sport', 'Science', 'Culture', 'IT', 'LifeSkills']
        news = []
        for category in categories:
            URLs = self.get_news_URLs_since(category, from_year, from_month)
            with tqdm(URLs) as pbar:
                pbar.set_description(f'[{category}] [Scraping news]')
                for URL in pbar:
                    news.append(self.parse_news(URL, category))
        news = list(filter(None, news))
        pd.DataFrame(news).to_csv(f'dataset.csv', encoding='utf-8')
        with zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.write('dataset.csv')
        os.remove('dataset.csv')

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Scraper ایجاد شده و شروع به دریافت و ذخیره‌سازی داده‌ها می‌کند. خبرهای دریافت شده همگی مربوط به قرن جدید، از سال ١٤٠٠ به بعد هستند.
    </p>
</div>

In [9]:
scraper = Scraper(current_year=1401, current_month=3)
#scraper.scrape(from_year=1400, from_month=1)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        بعد از ذخیره شدن داده‌ها در فایل فشرده dataset.zip، آن‌ها را از این فایل استخراج کرده و وارد برنامه می‌کنیم. با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [10]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    os.remove('dataset.csv')
    return dataset


dataset = pd.DataFrame(read_dataset_from_file())

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [11]:
def display_dataset_info():
    global CATEGORIES, dataset

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for _, data in dataset.iterrows():
        length_dict[data['category']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,68362
1,سیاسی,15798
2,جهانی,2895
3,اقتصادی,8900
4,اجتماعی,13585
5,شهری,3853
6,ورزشی,8348
7,علمی,3190
8,فرهنگی,6512
9,فناوری اطلاعات,437


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. این مراحل عبارتند از:
    </p>
</div>

<div dir="auto" align="justify">
    <li style='direction:rtl;text-align:justify;'>
        نرمال سازی داده‌ها (normalize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف لینک‌ها (remove_links)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف نشانه‌های نگارشی (remove_punctuations)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        واحد سازی داده‌ها (word_tokenize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف کلمات نامعتبر (remove_invalid_words)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف ایست‌واژه‌ها (remove_stopwords)
    </li>
</div>

In [12]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

In [13]:
def save_preprocessed_texts(texts, path="Preprocessed_texts.pickle"):
    with open(path, "wb") as file:
        pickle.dump(texts, file)


def load_preprocessed_texts(path="Preprocessed_texts.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)


def data_to_text(data):
    return ' '.join([data['title'], data['intro'], data['body']]).lower()


def get_preprocessed_texts(dataset, preprocessor, mode, save=False):
    preprocessed_texts = []
    if mode == 'process':
        texts = [data_to_text(data) for _, data in dataset.iterrows()]
        preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    if mode == 'load':
        preprocessed_texts = load_preprocessed_texts()
    if save:
        save_preprocessed_texts(preprocessed_texts)
    return preprocessed_texts

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Preprocessor ایجاد کرده و شروع به پیش پردازش داده‌ها می‌کنیم، یا داده‌های پیش‌پردازش‌شده‌ی ذخیره‌شده را از فایل مربوطه بازیابی می‌کنیم.
    </p>
</div>

In [17]:
preprocessor = Preprocessor(stopwords_path='stopwords.txt')

In [18]:
preprocessed_texts = get_preprocessed_texts(dataset, preprocessor, mode='load', save=False)

  0%|          | 0/4000 [00:14<?, ?it/s]


<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با توجه به این که حجم مجموعه داده‌ها باعث ایجاد محدودیت حافظه و زمان برای مدل tf-idf می‌شود، ناچار هستیم کسری از مجموعه‌ی داده‌ها را به عنوان ورودی برای یادگیری به آن بدهیم. با اجرای قطعه کد زیر، یک مجموعه داده‌ی کوچک‌تر از روی مجموعه داده‌ی اصلی ایجاد می‌کنیم.
    </p>
</div>

In [19]:
def get_mini_dataset(len_each_category=1000):
    global CATEGORIES, dataset

    mini_dataset = []
    for category in CATEGORIES.keys():
        dataset_by_category = dataset.loc[dataset['category'] == category]
        length = min(len_each_category, dataset_by_category.shape[0])
        mini_dataset.append(dataset_by_category.sample(length, random_state=1))

    mini_dataset = pd.concat(mini_dataset).reset_index(drop=True)
    texts = [data_to_text(data) for _, data in mini_dataset.iterrows()]
    mini_preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    return mini_dataset, mini_preprocessed_texts


# Almost 4K News
mini_dataset, mini_preprocessed_texts = get_mini_dataset(400)
mini_dataset['category_code'] = mini_dataset['category'].apply(lambda x: CATEGORIES_CLASSES[x])
labels = mini_dataset['category_code'].to_numpy()

100%|██████████| 4000/4000 [00:24<00:00, 163.58it/s]


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٣. دسته‌بندی اخبار بر اساس برچسب خبر
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این مرحله با استفاده از دو مدل متفاوت زبانی، یکی مبتنی بر روش سنتی رگرسیون لجستیک و دیگری بر پایه ترنسفرمر، شروع به یادگیری بر روی خبرهای استخراج شده می‌کنیم و در انتها، نتیجه اجرای ده کوئری را بر روی هر یک مشاهده می‌کنیم. در نهایت، ارزیابی‌های مورد بحث در داک تمرین را بر روی داده‌های تست انجام می‌دهیم.
    </p>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        کوئری‌های مورد استفاده در این بخش در زیر آمده‌اند.
    </p>
</div>

<div>
    <h4 style='direction:rtl;text-align:justify;'>
        ١. مدل مبتنی بر روش Logistic Regression
    </h4>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این بخش از کلاس TfidfVectorizer از کتاب‌خانه‌ی sklearn استفاده کرده و مجموعه کلمات یکتای موجود در عناوین خبرها را به عنوان ورودی به آن می‌دهیم. این کلمات یکتا را از روی فایل ذخیره شده‌ی vocabulary.pickle می‌خوانیم. در ادامه، یک مدل ایجاد کرده و بردار embedding عنوان هر خبر را به روش tf-idf به دست می‌آوریم. سپس، داده‌هایمان را به دو مجموعه train و test افراز می‌کنیم. حال، با استفاده از کلاس LogisticRegression کتاب‌خانه‌ی sklearn شروع به یادگیری داده‌های train می‌کنیم.
    </p>
</div>

In [20]:
THRESHOLD = 2

def relevant_text(bow_1, bow_2):
    bow_1 = set(bow_1)
    bow_2 = set(bow_2)
    len_bow1 = len(bow_1)
    len_bow2 = len(bow_2)
    same_words = bow_1.intersection(bow_2)
    len_same = len(same_words)
    if len_same >= min(len_bow1, len_bow2, THRESHOLD):
        return (len_same/len_bow1), (len_same/len_bow2)
    else:
        return 0, 0

In [ ]:
adjacency_matrix = np.zeros(shape=(mini_dataset.shape[0], mini_dataset.shape[0]))

In [34]:
titles = list(mini_dataset['intro'])
for i in range(len(titles)):
    titles[i] = preprocessor.preprocess(titles[i])

In [35]:
for i in tqdm(range(mini_dataset.shape[0]), position=0, leave=True):
    for j in range(i, mini_dataset.shape[0]):
        if i == j:
            continue
        edge_j_i, edge_i_j = relevant_text(titles[i], titles[j])
        adjacency_matrix[i, j] = edge_i_j
        adjacency_matrix[j, i] = edge_j_i

100%|██████████| 4000/4000 [00:29<00:00, 134.03it/s]


In [23]:
adjacency_matrix = normalize(adjacency_matrix, norm='l1')
G = nx.from_numpy_array(adjacency_matrix)

In [24]:
pr = nx.pagerank(G)

In [25]:
vals = np.array(list(pr.values()))
idx = np.argsort(-vals)
mini_dataset.iloc[idx[:5]]

,date,title,intro,body,category,category_code
2776,دوشنبه ۱۷ آبان ۱۴۰۰ - ۱۷:۳۰,متاورس چیست؟,متاورس (Metaverse) یک واژه‌ مرکب متشکل از met...,این بار زاکربرگ سعی کرده به یکی از تصورات و خی...,Science,6
903,دوشنبه ۶ دی ۱۴۰۰ - ۱۳:۱۶,جزئیات جدید از طرح بنزینی دولت | زمان اجرای طر...,وزیر نفت گفت: دولت سیزدهم برنامه‌ای برای افزای...,به گزارش همشهری آنلاین به نقل از باشگاه خبرنگا...,Economy,2
733,پنجشنبه ۵ اسفند ۱۴۰۰ - ۱۰:۲۸,ببینید | اوکراین؛ از آژیر حمله هوایی تا آتش سو...,رئیس جمهوری روسیه صبح امروز اعلام کرد که در پا...,به گزارش همشهری آنلاین به نقل از تسنیم، شورای ...,World,1
1212,چهارشنبه ۱۵ اردیبهشت ۱۴۰۰ - ۱۵:۴۸,اینفوگرافیک | روند کرونا در ایران از ۱۵ فروردی...,روند شیوع کرونا را در ایران طی یک ماه اخیر (از...,به گزارش همشهری‌آنلاین به نقل از ایسنا، ویروس ...,Society,3
1554,سه‌شنبه ۳۰ آذر ۱۴۰۰ - ۰۰:۲۸,پلیس، زندانیان قدیمی شکارچی دختران فراری را شن...,«یک هفته به همراه دخترم در خانه وحشت زندانی بو...,به گزارش همشهری آنلاین، رسیدگی به پرونده خانه ...,Society,3


In [26]:
h, a = nx.hits(G)

In [32]:
vals = np.array(list(h.values()))
idx = np.argsort(-vals)
mini_dataset.iloc[idx[:5]]

,date,title,intro,body,category,category_code
2776,دوشنبه ۱۷ آبان ۱۴۰۰ - ۱۷:۳۰,متاورس چیست؟,متاورس (Metaverse) یک واژه‌ مرکب متشکل از met...,این بار زاکربرگ سعی کرده به یکی از تصورات و خی...,Science,6
3428,سه‌شنبه ۱۲ بهمن ۱۴۰۰ - ۱۰:۰۰,چهارشنبه مهم برای علاقه‌مندان به متاورس,از ماه اکتبر که فیسبوک نام خود را به متا تغییر...,به گزارش همشهری آنلاین و به نقل از خبرگزاری رو...,IT,8
3565,یکشنبه ۱۱ اردیبهشت ۱۴۰۱ - ۱۹:۳۳,سود ۸ تریلیون تومانی در متاورس | میمونی که کسل...,شرکت تولیدکننده سری توکن‌های غیرمثلی «میمون‌ ک...,به گزارش همشهری آنلاین و به نقل از رویترز، سال...,IT,8
3470,یکشنبه ۲۶ دی ۱۴۰۰ - ۱۴:۳۸,تماشای رقابت‌های NBA در متاورس | گامی مهم در ت...,انجمن ملی بسکتبال (NBA) در دستگاه‌های ۲۹۹ دلار...,به گزارش همشهری آنلاین و به نقل از سی‌ان‌بی‌سی...,IT,8
3486,شنبه ۳۱ اردیبهشت ۱۴۰۱ - ۰۸:۵۸,صف مخالفان متاورس طولانی‌تر شد | نقدی بر ایده‌...,مدیر سخت‌افزار شرکت آمازون در یک رویداد عمومی ...,به‌گزارش همشهری آنلاین و به نقل از وب‌سایت ورج...,IT,8


In [33]:
vals = np.array(list(a.values()))
idx = np.argsort(-vals)
mini_dataset.iloc[idx[:5]]

,date,title,intro,body,category,category_code
2776,دوشنبه ۱۷ آبان ۱۴۰۰ - ۱۷:۳۰,متاورس چیست؟,متاورس (Metaverse) یک واژه‌ مرکب متشکل از met...,این بار زاکربرگ سعی کرده به یکی از تصورات و خی...,Science,6
3428,سه‌شنبه ۱۲ بهمن ۱۴۰۰ - ۱۰:۰۰,چهارشنبه مهم برای علاقه‌مندان به متاورس,از ماه اکتبر که فیسبوک نام خود را به متا تغییر...,به گزارش همشهری آنلاین و به نقل از خبرگزاری رو...,IT,8
3565,یکشنبه ۱۱ اردیبهشت ۱۴۰۱ - ۱۹:۳۳,سود ۸ تریلیون تومانی در متاورس | میمونی که کسل...,شرکت تولیدکننده سری توکن‌های غیرمثلی «میمون‌ ک...,به گزارش همشهری آنلاین و به نقل از رویترز، سال...,IT,8
3470,یکشنبه ۲۶ دی ۱۴۰۰ - ۱۴:۳۸,تماشای رقابت‌های NBA در متاورس | گامی مهم در ت...,انجمن ملی بسکتبال (NBA) در دستگاه‌های ۲۹۹ دلار...,به گزارش همشهری آنلاین و به نقل از سی‌ان‌بی‌سی...,IT,8
3486,شنبه ۳۱ اردیبهشت ۱۴۰۱ - ۰۸:۵۸,صف مخالفان متاورس طولانی‌تر شد | نقدی بر ایده‌...,مدیر سخت‌افزار شرکت آمازون در یک رویداد عمومی ...,به‌گزارش همشهری آنلاین و به نقل از وب‌سایت ورج...,IT,8


<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این قسمت، دسته‌ی هر یک از کوئری‌های مشخص شده را با استفاده از مدل یادگیری پیش‌بینی می‌کنیم.
    </p>
</div>

In [29]:
def dump_all():
    dict_of_all = {
        'mini_dataset' : mini_dataset,
        'mini_preprocessed_texts': mini_preprocessed_texts,
        'adjacency_matrix': adjacency_matrix,
        'nx_graph': G,
        'pagerank': pr,
        'hub': h,
        'authority': a
    }
    with open('dict_all.pickle', 'wb') as fp:
        pickle.dump(dict_of_all, fp)

In [30]:
dump_all()